# Multi-Scenario MORDM

Multi-scenario MORMD is an extension of normal MORDM to better include robustness considerations within the search phase. It starts from the scenario discovery results resulting from MORDM. Next, from the experiments within this box, a set of scenarios is selected. 

There are many ways of selecting the additional scenarios. The original paper which introduced multi-scenario MORMD [Watson and Kaspzryk (2017)](https://doi.org/10.1016/j.envsoft.2016.12.001) did it in a more or less adhoc manner. [Eker and Kwakkel (2018)](https://doi.org/10.1016/j.envsoft.2018.03.029) introduced a more formal selection approach, the code of which can be found on [GitHub](https://github.com/sibeleker/MORDM---Multi-scenario-search). 

For this assignment, make an informed selection of 4 scenarios, using an approach of your choice. Motivate carefully your selection procedure. 


In [18]:
from dps_lake_model import lake_model
from ema_workbench import (Model, RealParameter, ScalarOutcome,
                           MultiprocessingEvaluator, ema_logging,
                           Constant, Constraint)
from ema_workbench.em_framework import samplers
from ema_workbench.em_framework import sample_levers
from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)

# Instantiate the model
lake_model = Model('lakeproblem', function=lake_model)
lake_model.time_horizon = 100 # used to specify the number of timesteps

# Specify uncertainties
lake_model.uncertainties = [RealParameter('mean', 0.01, 0.05),
                            RealParameter('stdev', 0.001, 0.005),
                            RealParameter('b', 0.1, 0.45),
                            RealParameter('q', 2.0, 4.5),
                            RealParameter('delta', 0.93, 0.99)]

lake_model.levers = [RealParameter('c1', -2, 2),
                            RealParameter('c2', -2, 2),
                            RealParameter('r1', 0, 2),
                            RealParameter('r2', 0, 2),
                            RealParameter('w1', 0, 1)]
# Specify outcomes
lake_model.outcomes = [ScalarOutcome('max_P', kind=ScalarOutcome.MINIMIZE, expected_range=(0,5)), #the bounds on max_P are not known up front. introduce a constraint saying that max_P must be below 5.
                       ScalarOutcome('utility', kind=ScalarOutcome.MAXIMIZE,expected_range=(0,2)),
                       ScalarOutcome('inertia', kind=ScalarOutcome.MAXIMIZE,expected_range=(0,1)),
                       ScalarOutcome('reliability', kind=ScalarOutcome.MAXIMIZE,expected_range=(0,1))]

# Run scenario
# Generate policies by sampling over the levers (generate 5 sample levers)
test_policies = sample_levers(lake_model,n_samples=5)
nr_experiments = 1000

print(test_policies)
       

ema_workbench.DefaultDesigns, 1 designs on 5 parameters


# Scenario selection

Possible criteria to select a small number of scenarios from a large set are internal consistency, diversity of outcome indicators, extremeness, and policy relevance (Trutnevyte et al., 2016). We would like to use the formal selection approach from Eker and Kwakkel (2018), which selects maximum diverse scenarios.

The following code is partially based on https://github.com/sibeleker/MORDM---Multi-scenario-search/blob/master/MORDM_paper/MORDM%20with%20multi-scenario%20search-Rev_v1.ipynb

However, there might be a misconception (from Eker and Kwakkel (2018), we assumed that the scenario selection takes place before running MORDM, as it is the first step in the MORDM with multi-scenario search(DPR).ipynb workbook).

*We assume the correct order is:*

* Select four scenarios from the output of Assignment 8, Step 3 (Re-evaluate candidate solutions under uncertainty).
* The scenarios from this re-evaluation of candidate solutions are then used to find new policies taking into account the uncertainties of the scenarios.
* Given the above applies, does it matter which policy has lead to the scenario, or are only the uncertainties relevant to establish a scenario in this context, and new policies are found in the second round of MORDM?


## Specify model, run N=1000 scenarios

In [23]:
with MultiprocessingEvaluator(lake_model) as evaluator:
    results = evaluator.perform_experiments(lake_model, nr_experiments, policies=test_policies)

TypeError: perform_experiments() got multiple values for argument 'policies'

We do not understand the error. Based on https://emaworkbench.readthedocs.io/en/latest/ema_documentation/em_framework/evaluators.html, the policies argument expects either int or collection of Policy instances. In this case, policies is a collection of Policy instances.

## Select policy relevant scenarios based on exploration results

The requirements are taken from Eker and Kwakkel (2018).
* max_P >= median(~ 5)
* reliability <= median (~0.5)
* inertia <= median (~1)
* utility <= median (~0.15)

Only scenarios above the outcome median should be selected. Otherwise, they are not considered relevant.

In [24]:
# Based on https://github.com/sibeleker/MORDM---Multi-scenario-search/blob/master/MORDM_paper/MORDM%20with%20multi-scenario%20search-Rev_v1.ipynb

experiments, outcomes = results
oois = sorted(outcomes.keys())

#here, the policy-relevant scenarios defined by median thresholds are selected
indices = []
for ooi in oois:
    if ooi in ['max_P', 'inertia']:
        a = outcomes[ooi] > np.median(outcomes[ooi])     
    else: 
        a = outcomes[ooi] < np.median(outcomes[ooi])
    indices.append(a)
indices = np.swapaxes(indices, 0, 1)
logical_index = np.array([index.all() for index in indices])
newExperiments = experiments[logical_index]
newOutcomes = {}
for ooi in oois:
    newOutcomes[ooi] = outcomes[ooi][logical_index]
newResults = newExperiments, newOutcomes

NameError: name 'results' is not defined

## Find four maximimally diverse scenario in the subset of policy relevant scenarios
This is performed by exhaustive search

In [ ]:
# Based on https://github.com/sibeleker/MORDM---Multi-scenario-search/blob/master/MORDM_paper/MORDM%20with%20multi-scenario%20search-Rev_v1.ipynb

import pandas as pd
import copy
#0 : non policy-relevant scenarios
#1 : policy-relevant scenarios
#2 : prim results
#3 : diverse SELECTED
#4 : random selected

sel_column = logical_index.astype(int)
#selected = [153, 160, 197, 207] #========SELECTED=================
selected = [77,  96, 130, 181]
random_selected = [81, 289, 391, 257]
count = 0
for index, i in enumerate(sel_column):
    
    if prim_logical_index[index]: #this is computed at the bottom of  the notebook
        sel_column[index] = 2
    if i:
        if count in selected:
            sel_column[index] = 3
        count +=1 #the reason for the count is that the selected indices correspond to the dataset of 206 scenarios 
            
    if index in random_selected:
        sel_column[index] = 4
        

print(len(sel_column))

## Search for each scenario

For each of the four selected scenarios, use many-objective optimization to find a pareto approximate set using the same approach as for assignment 8. Remember to check for convergence (and time permitting, seed analysis), and be careful in what epsilon values to use (not to coarse, not too small). 

Store the resulting set of pareto solutions in a smart way for subsequent analysis.


## Re-evaluate under deep uncertainty

Combine the pareto set of solutions found for each scenario. Next, turn each solution into a policy object. If you have a very large number of policies, you can choose to down sample your policies in some reasoned way (*e.g.*, picking min and max on each objective, slicing across the pareto front with a particular step size). As a rule of thumb, try to limit the set of policies to at most 50. 

Re-evaluate the combined set of solutions over 1000 scenarios sampled using LHS.


Calculate both the maximum regret, and the domain criterion using the values provided in [Bartholomew and Kwakkel (2020)](https://doi.org/10.1016/j.envsoft.2020.104699). Ignore the max_P objective.

visualize the results in parallel coordinate plot. 

Are there any promising compromise solutions which balance performance in both the reference scenarios as well as in terms of their robustness?
